# Libraries

In [1]:
# Initiate server
#
# $ docker-compose up

In [2]:
# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# Basic libraries
#
import pickle
import pandas as pd
import numpy as np
import time
import os
import gc


# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# Scipy library
#
from scipy import signal


# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# Triton client libraries
#
import tritonclient
import tritonclient.grpc                  as triton_grpc  
import tritonclient.grpc.model_config_pb2 as mc
from   tritonclient.utils                 import triton_to_np_dtype
from   tritonclient.utils                 import InferenceServerException


# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# User libraries
#
from utils.Logger        import *
from utils.InfuxDBClient import *

# Parameters

In [3]:
HOST    = 'localhost'
PORT    = 6001
TIMEOUT = 60


PATH     = os.path.abspath('model_repository')


# Model names
#
model_name  = 'AnomalyDetectionModel'


# Input & output names as well as models' version is the same
#
input_name    = 'InputLayer'
output_name   = 'OutputLayer'
model_version = '1'


# Selected sensor
#
Sensor = '01cc0529-79bb-48cc-a052-4fde9372abeb'

# Set look-back (Lag)
#
Lag = 32

# Set 'verbose'
#
VERBOSE = True


url = '{}:{}'.format(HOST, PORT )

## Initiate logger

In [4]:
# Initiate logger
#
if VERBOSE:
    logger = init_logger( log_file = 'logs.log' )  

# Triton Serving

In [5]:
### ------------------------------------------------------------------------- ### 
## setup TRT
### ------------------------------------------------------------------------- ### 
#
triton_client  = triton_grpc.InferenceServerClient(url     = url, 
                                                   verbose = VERBOSE)


# Wait for server to come online
server_start = time.time()
while True:
    try:
        if triton_client.is_server_ready() or time.time() - server_start > TIMEOUT:
            if VERBOSE: 
                logger.info('[INFO] TRITON server is ready')
            break
        
    except InferenceServerException:
        if VERBOSE: 
            logger.error('[ERROR] TRITON server is not ready')
        pass
   
    time.sleep(1)

[INFO] TRITON server is ready


is_server_ready, metadata ()

ready: true



# Data

## Download data from InfuxDB

## Setup InfuxDB Client

In [6]:
# Time limit
#
LIMIT = 10 * 32


influx_measurement = "beat_box_rms"


# Change to influx live data 
#
influx = InfluxDataframeDBClient(host     = '192.168.2.198', 
                                 username = 'admin',
                                 pwd      = 'CoreInn02019', 
                                 port     =  8086, 
                                 db       = 'corebeat_rms')
influx.initClient()

### Download data

In [7]:
# Start timer
#
start_timer = time.time()

# TABLE: corebeat_rms
#
df = influx.client.query(f'SELECT * from {influx_measurement} order by time DESC limit {LIMIT}') 
df = df[ influx_measurement ]
#
if VERBOSE: 
    logger.info('[INFO] Download data from CoreBEAT database')
    logger.info('[INFO] Time: {:.2f}s'.format(time.time() - start_timer))



# Preprocess data
#
df = df[ df['beat_box_id'] == Sensor]

# 
del df['beat_box_id']


# Get only the last -Lag- required measurements
#
df = df[-Lag:]


if VERBOSE: 
    logger.info('[INFO] Data cleaning - Data processing')
    logger.info('[INFO] Number of features: {}'.format(df.shape[1]))
    
df.head( 3 )

[INFO] Download data from CoreBEAT database
[INFO] Time: 0.04s
[INFO] Data cleaning - Data processing
[INFO] Number of features: 4


,acc_x,acc_y,acc_z,temperature
2022-07-07 10:50:01+00:00,0.574637,0.701521,0.957487,37.3
2022-07-07 10:51:00+00:00,0.194573,0.171073,0.206829,37.3
2022-07-07 10:52:00+00:00,0.228922,0.227197,0.343287,37.4


## Create instance for inference

In [8]:
Instance = df.to_numpy()

if VERBOSE: 
    logger.info('[INFO] Instance created')
    logger.info('[INFO] Number of features: {}'.format(Instance.shape[1]))

[INFO] Instance created
[INFO] Number of features: 4


## Pre-process

In [9]:
def Preprocess( Instance, model_name ):
    
    # Load Scaler
    #
    scaler = pickle.load( open( '{}/{}/Scaler.pkl'.format(PATH, model_name), 'rb'))    
    
    # Apply scaler
    #
    Instance = scaler.transform( Instance )
    
    # Add dimension
    #
    Instance = np.expand_dims(Instance, axis=0)
    
    return Instance

In [10]:
Instance = Preprocess( Instance, model_name )


if VERBOSE: 
    logger.info('[INFO] Instance was preprocessed')

/Users/livieris/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
[INFO] Instance was preprocessed


# Inference

In [11]:
def inference(data, model_name, input_name, output_name, verbose = False):

    # Setup inputs for inference
    #
    modelInputs = triton_grpc.InferInput(input_name, shape=[1, 32, 4], datatype='FP32');
    modelInputs.set_data_from_numpy( data.astype('float32') );
    
    
    # Setup output for inference
    #
    modelOutputs = triton_grpc.InferRequestedOutput( output_name );

    
    # Get response
    #
    response = triton_client.infer(model_name, 
                                   model_version = model_version, 
                                   inputs        = [ modelInputs  ], 
                                   outputs       = [ modelOutputs ]);
    

    # Process response
    #
    logits = response.as_numpy( output_name );
    logits = np.asarray(logits, dtype=np.float32);

    if (verbose):
        logger.info(f"[INFO] Infer with TRITON server: {model_name}");
        logger.info(f"[INFO] Prediction shape: {logits.shape} ");
        
        
        
    # Get predictions
    #
    pred    = np.array(logits)

    
    # Calculate losses
    #
    mae_loss = np.mean( np.mean(np.abs(pred - data), axis = 2), axis = 1)
    mse_loss = np.mean(np.mean((pred - data)**2, axis=-1), axis = 1)
    if (verbose):
        logger.info('[INFO] MAE & MSE loss calculated')



        
    # Clean garbage
    #
    del modelInputs, response
    del logits
    gc.collect();


    
    

    # Create DataFrame with output/response
    #
    df_response = pd.DataFrame()
    
    # Store losses
    #
    df_response['MSE loss'] = mse_loss
    df_response['MAE loss'] = mae_loss

    if (verbose): 
        logger.info(f"Output df: {df_response.head(3)} ")
    
    
    
    return df_response

In [12]:
if ( np.isnan(Instance).any() ):
    if VERBOSE: logger.error('[ERROR] Instance contained NaN values')
else:
    df_response = inference(data          = Instance, 
                            model_name    = model_name, 
                            input_name    = input_name,
                            output_name   = output_name,
                            verbose       = VERBOSE);
    
    if VERBOSE: logger.info('[INFO] Predictions performed')




infer, metadata ()
model_name: "AnomalyDetectionModel"
model_version: "1"
inputs {
  name: "InputLayer"
  datatype: "FP32"
  shape: 1
  shape: 32
  shape: 4
}
outputs {
  name: "OutputLayer"
}
raw_input_contents: ",\240\306@\311\252\013A\224\350\342@\2401\033@2B2=W\204\224\272 \317\256=\2401\033@\252\276\031?=.l?\326\356\255?\334i @r\266$\277M@\374\276\221X\222\275\2401\033@\326 (\277\372\341\317\276c\177\362\275\334i @\376\207\303@{{\016A9.\005A\2401\033@g\224\014\277|,;\275\212{\321\275\2401\033@\205H\204?\270\037\370?(e\355?\2401\033@\340\373&\277\235\237\252\276&~ >\2401\033@\377\362\217\277\0312x\277\354+\024\277\334i @\330\374\264@\026A\322@\363O\337@\2401\033@:sc\276X\r\313=1(\215>\2401\033@\265\234<\276m\007?\276\365\345\331=(\301\020@8>\276?\212\263\371?k\310\375?(\301\020@\251\370_:\\o\231<\210\263H\273d\371\025@\255\020X?@b\306?\375\212\230?d\371\025@\351EL\275\312\032\216\275\'\352{\276d\371\025@\351\300A@\036\232[@.\362U@d\371\025@\313%5@\024AL@\275\351l@\2401\033@.\337\25

[INFO] Infer with TRITON server: AnomalyDetectionModel
[INFO] Prediction shape: (1, 32, 4) 
[INFO] MAE & MSE loss calculated
Output df:    MSE loss  MAE loss
0  2.030342  1.091005 
[INFO] Predictions performed


model_name: "AnomalyDetectionModel"
model_version: "1"
outputs {
  name: "OutputLayer"
  datatype: "FP32"
  shape: 1
  shape: 32
  shape: 4
}
raw_output_contents: "\236\303\222@UW\270@)\023\274@\376\353\326?Y\"B?:\351z?)\270r?\366\007\305?\345?]\2761v\227\27690\002\277\320|\316?\256|\013\276\302\365J\276h\265\265\2766U\263?\227,\264=\222\322\353>\362y\263>y\033\257?\022\311\236@(h\302@H&\326@\330\361\205?%\262A\276V&\210=R\261\260>\251\237\214?\234G\202>\262-+?\276\0356?\376\243\245?\2202?\277J\317\215\275pj\256\276\037\334\231?\326#\276\273\244c\010?c\303i>R\'}?\022\307\223@A\253\260@\3731\271@i\022r?D\340!\277X\223\325\274\264Z\273\276U\365\216?\267@\224\276x\264\376\274\257\253p\276te\274?\266\3563>\235)\235>\355\0228?\n\312\305?\362w&?\356;}?h\351\234?\367|\245?\000T\266>a\267.?\354\353k?\217}\273?\251\233T?6\341\252?\010q\265?\014`\267?\334ii?\267\036\272?\355\356\264?K\177\223?F\301\023@[K;@\313\2046@@\216\222?\362$\213@$\307\243@wP\240@\324\273\302?\327\355\222?\233\262\221?\362